<a href="https://colab.research.google.com/github/bariskucuk1999/data-science-notes/blob/main/%C3%96dev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
veriler=pd.read_csv("odev_tenis.txt")
veriler

,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


In [ ]:
from sklearn import preprocessing #categoric--->numeric
veriler2=veriler.apply(preprocessing.LabelEncoder().fit_transform) #tek satırda encoding işlemleri tüm column'lara uygulandı
veriler2

,outlook,temperature,humidity,windy,play
0,2,11,4,0,0
1,2,8,6,1,0
2,0,10,5,0,1
3,1,4,9,0,1
4,1,2,3,0,1
5,1,1,1,1,0
6,0,0,0,1,1
7,2,6,8,0,0
8,2,3,1,0,1
9,1,7,3,0,1


In [ ]:
c=veriler2.iloc[:,:1]
ohe=preprocessing.OneHotEncoder() #column başlıklarını etiketlere taşıma ve her etiketin altında 0 veya 1 aitliğini belirleme
c=ohe.fit_transform(c).toarray()
c #column başlıkları değiştirilmiş durumda(overcast,rainy ve sunny nümerik hale getirildi)

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [ ]:
havadurumu=pd.DataFrame(data=c,index=range(14),columns=["overcast","rainy","sunny"])
sonveriler=pd.concat([havadurumu,veriler.iloc[:,1:3]],axis=1) #hava durumu ile diğer verileri birleştirme

,overcast,rainy,sunny,temperature,humidity
0,0.0,0.0,1.0,85,85
1,0.0,0.0,1.0,80,90
2,1.0,0.0,0.0,83,86
3,0.0,1.0,0.0,70,96
4,0.0,1.0,0.0,68,80
5,0.0,1.0,0.0,65,70
6,1.0,0.0,0.0,64,65
7,0.0,0.0,1.0,72,95
8,0.0,0.0,1.0,69,70
9,0.0,1.0,0.0,75,80


In [ ]:
sonveriler=pd.concat([veriler2.iloc[:,-2:],sonveriler],axis=1)
sonveriler
#windy ve play ---> label encoding kullanıldı
#overcast,rainy ve sunny ---> onehot encoding kullanıldı

,windy,play,overcast,rainy,sunny,temperature,humidity
0,0,0,0.0,0.0,1.0,85,85
1,1,0,0.0,0.0,1.0,80,90
2,0,1,1.0,0.0,0.0,83,86
3,0,1,0.0,1.0,0.0,70,96
4,0,1,0.0,1.0,0.0,68,80
5,1,0,0.0,1.0,0.0,65,70
6,1,1,1.0,0.0,0.0,64,65
7,0,0,0.0,0.0,1.0,72,95
8,0,1,0.0,0.0,1.0,69,70
9,0,1,0.0,1.0,0.0,75,80


In [ ]:
from sklearn.model_selection import train_test_split #verinin eğitim ve test için bölünmesi
x_train, x_test, y_train, y_test=train_test_split(sonveriler.iloc[:,:-1],sonveriler.iloc[:,-1:],test_size=0.33,random_state=0)
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(x_train,y_train) #x_train'den y_train'i öğrenme
#(x_train->humidity hariç tablodaki veriler, y_train->humidity column'u)
#x_train ve y_train verinin %67'lik kısmı, (test_size=0.33 aldık)
#x_test ve y_test verinin %33'lük kısmı
y_pred=regressor.predict(x_test) 
y_pred

array([[84.45365573],
       [63.93839954],
       [85.76050662],
       [64.21013241],
       [75.06793322]])

In [ ]:
y_test #y_pred ile karşılaştırırsak;

,humidity
8,70
6,65
4,80
11,90
2,86


In [ ]:
#backward elimination ile tahmini iyileştirmek istersek;
import statsmodels.api as sm 
X=np.append(arr=np.ones((14,1)).astype(int),values=sonveriler.iloc[:,:-1],axis=1)
X_l=sonveriler.iloc[:,[0,1,2,3,4,5]].values #tüm column'ları(6 column) aldık (bağımsız değişkenler)
X_l=np.array(X_l,dtype=float)
model=sm.OLS(sonveriler.iloc[:,-1:],X_l).fit() #bağımlı değişken(humidity) ile bağımsız değişkenleri içeren dizi arasında bağlantı
model.summary() #p-value'ları bulmak için

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               humidity   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                 -0.148
Method:                 Least Squares   F-statistic:                    0.6653
Date:                Wed, 02 Mar 2022   Prob (F-statistic):              0.661
Time:                        15:39:44   Log-Likelihood:                -49.542
No. Observations:                  14   AIC:                             111.1
Df Residuals:                       8   BIC:                             114.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -4.0286      7.229     -0.557      0.593     -20.698      12.641
x2            -8.2778      8.029     -1.031      0.333     -26.793      10.237
x3            52.3891     50.214      1.043      0.327     -63.404     168.183
x4            55.6337     45.233      1.230      0.254     -48.673     159.940
x5            49.4291     48.337      1.023      0.336     -62.035     160.893
x6             0.4920      0.597      0.825      0.433      -0.884       1.868
==============================================================================
Omnibus:                        0.935   Durbin-Watson:                   2.416
Prob(Omnibus):                  0.627   Jarque-Bera (JB):                0.823
Skew:                           0.389   Prob(JB):                        0.663
Kurtosis:                       2.103   Cond. No.                     2.08e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.08e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#x1'in p-value'su en yüksek olduğundan x1'i eleriz
sonveriler=sonveriler.iloc[:,1:]
X=np.append(arr=np.ones((14,1)).astype(int),values=sonveriler.iloc[:,:-1],axis=1)
X_l=sonveriler.iloc[:,[0,1,2,3,4]].values
X_l=np.array(X_l,dtype=float)
model=sm.OLS(sonveriler.iloc[:,-1:],X_l).fit() #bağımlı değişken(humidity) ile bağımsız değişkenleri içeren dizi arasında bağlantı
model.summary()

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               humidity   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                 -0.060
Method:                 Least Squares   F-statistic:                    0.8165
Date:                Wed, 02 Mar 2022   Prob (F-statistic):              0.546
Time:                        15:45:39   Log-Likelihood:                -49.809
No. Observations:                  14   AIC:                             109.6
Df Residuals:                       9   BIC:                             112.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -6.2865      6.909     -0.910      0.387     -21.916       9.343
x2            36.6564     39.905      0.919      0.382     -53.615     126.927
x3            41.9135     36.464      1.149      0.280     -40.573     124.400
x4            35.1064     39.339      0.892      0.395     -53.884     124.097
x5             0.6484      0.506      1.282      0.232      -0.496       1.793
==============================================================================
Omnibus:                        0.887   Durbin-Watson:                   2.360
Prob(Omnibus):                  0.642   Jarque-Bera (JB):                0.810
Skew:                           0.424   Prob(JB):                        0.667
Kurtosis:                       2.181   Cond. No.                     1.73e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#x_train ve x_test'ten ilk column'u(windy) atmanın yeterli olduğu görülmüştür
x_train=x_train.iloc[:,1:]
x_test=x_test.iloc[:,1:]
regressor.fit(x_train,y_train) #x_train'den y_train'i öğrenme
#(x_train->humidity hariç tablodaki veriler, y_train->humidity column'u)
#x_train ve y_train verinin %67'lik kısmı, (test_size=0.33 aldık)
#x_test ve y_test verinin %33'lük kısmı
y_pred=regressor.predict(x_test) 
y_pred

array([[77.98135142],
       [68.29304916],
       [81.05037539],
       [71.44926132],
       [75.78905304]])

In [ ]:
y_test #y_pred ile karşılaştırırsak; önceki tahmine göre daha iyi bir tahmin olduğu görülmüştür

,humidity
8,70
6,65
4,80
11,90
2,86
